# Part I: Demand Unconstraining: Physical products with limited inventory 
* Dataset: grocery_data

## a. Double Exponential Smoothing (DES) estimations

- Uses the unconstrained part of the constrained curve to make prediction
- Try (alpha,beta) = 0.1 to 1 using 0.1 increments and forecast
- Assume a burn-in period of 3 hours

In [1]:
# import libraries
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.api import ExponentialSmoothing
from itertools import product
! pip install "statsmodels==0.12.0
! pip install statsmodels --upgrade

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [2]:
# load grocery data
import pandas as pd
df_grocery = pd.read_excel("/content/drive/MyDrive/BA_875/Datasets/grocery_data.xlsx")
df_grocery.info()
df_grocery

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       21 non-null     object 
 1   Hourly interval  21 non-null     int64  
 2   Unnamed: 2       21 non-null     int64  
 3   Unnamed: 3       21 non-null     int64  
 4   Unnamed: 4       21 non-null     int64  
 5   Unnamed: 5       21 non-null     int64  
 6   Unnamed: 6       21 non-null     int64  
 7   Unnamed: 7       21 non-null     int64  
 8   Unnamed: 8       21 non-null     int64  
 9   Unnamed: 9       20 non-null     float64
 10  Unnamed: 10      20 non-null     float64
 11  Unnamed: 11      17 non-null     float64
 12  Unnamed: 12      17 non-null     float64
 13  Unnamed: 13      15 non-null     float64
 14  Unnamed: 14      15 non-null     float64
 15  Unnamed: 15      11 non-null     float64
 16  Unnamed: 16      8 non-null      float64
 17  Total            2

,Unnamed: 0,Hourly interval,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Total
0,Day,1,2,3,4,5,6,7,8,9.0,10.0,11.0,12.0,13.0,14.0,15.0,16.0,NaN
1,1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN,40.0
2,2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
3,3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,40.0
4,4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN,40.0
5,5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
6,6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0
7,7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,40.0
8,8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0
9,9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0


In [3]:
# remove first row and first column 
# now each row is a grocery demand curve
df_grocery = df_grocery.iloc[1:,1:]
df_grocery.head(20)

,Hourly interval,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Total
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN,40.0
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,40.0
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN,40.0
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0
7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,40.0
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0
10,2,6,6,3,4,2,4,2,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0


In [4]:
# rename columns
import numpy as np
df_grocery.columns = np.arange(1,18)
#df_grocery = df_grocery.add_suffix("hour")
df_grocery.rename(columns={17: 'Total'}, inplace = True)
df_grocery.head(20)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Total
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN,40.0
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,40.0
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN,40.0
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0
7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,40.0
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0
10,2,6,6,3,4,2,4,2,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0


### Convert to the cumulative demand curve

In [5]:
df_grocery_c = df_grocery.iloc[:,:16]
df_grocery_c = np.cumsum(df_grocery_c,axis=1)
df_grocery_c

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,40.0,NaN,NaN,NaN,NaN
2,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,40.0,NaN,NaN
4,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,40.0,NaN
5,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,4.0,7.0,8.0,11.0,14.0,16.0,17.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0
7,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,40.0,NaN,NaN,NaN,NaN
8,1.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,12.0,13.0,15.0,16.0,17.0,18.0,19.0,20.0
9,1.0,3.0,6.0,8.0,10.0,12.0,14.0,15.0,17.0,19.0,21.0,23.0,25.0,28.0,30.0,31.0
10,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN


### Step 1: Tease out the constrained booking curves

In [6]:
df_grocery_c[df_grocery_c.isnull().any(axis = 1)]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,40.0,NaN,NaN,NaN,NaN
2,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,40.0,NaN,NaN
4,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,40.0,NaN
5,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
7,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,40.0,NaN,NaN,NaN,NaN
10,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
12,2.0,5.0,9.0,11.0,13.0,14.0,16.0,18.0,21.0,23.0,26.0,30.0,33.0,36.0,39.0,NaN
13,2.0,5.0,7.0,8.0,11.0,13.0,16.0,18.0,20.0,21.0,22.0,24.0,27.0,31.0,NaN,NaN
14,1.0,5.0,8.0,9.0,11.0,12.0,15.0,17.0,20.0,22.0,26.0,31.0,37.0,40.0,NaN,NaN


In [7]:
# check which curves are censored, create boolean variable identifying censored cases
# if there is any nan value appeared in the cumulative demand dataset, that row we think is censored
df_grocery_c['censored'] = df_grocery_c.isnull().any(axis = 1)
# now that we have identified the censored curves, we need to uncensor the data
df_grocery_c['censored']

1      True
2      True
3      True
4      True
5      True
6     False
7      True
8     False
9     False
10     True
11    False
12     True
13     True
14     True
15     True
16    False
17     True
18    False
19     True
20    False
Name: censored, dtype: bool

### Step 2: apply DES to the constrained curves; find best alpha and beta for each; predict

In [8]:
## DES method for demand uncensoring 
# copy the censored series (rows)
df_des = df_grocery_c[df_grocery_c.censored].drop('censored',axis=1)
df_des.reset_index(inplace = True)
df_des.drop(columns="index", inplace = True)
df_des

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,40.0,NaN,NaN,NaN,NaN
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,40.0,NaN,NaN
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,40.0,NaN
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,40.0,NaN,NaN,NaN,NaN
6,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,5.0,9.0,11.0,13.0,14.0,16.0,18.0,21.0,23.0,26.0,30.0,33.0,36.0,39.0,NaN
8,2.0,5.0,7.0,8.0,11.0,13.0,16.0,18.0,20.0,21.0,22.0,24.0,27.0,31.0,NaN,NaN
9,1.0,5.0,8.0,9.0,11.0,12.0,15.0,17.0,20.0,22.0,26.0,31.0,37.0,40.0,NaN,NaN


#### Replace each Censored Demand values from all censored demend curves with NaN 

In [9]:
# replace each Censored Demand values from all censored demend curves with NaN, 
# as we will use the unconstrained part of the constrained curves to make our prediction
# adjust the grocery data of sales by removing the sales possibly affected by out-of-stock
# check at which hour the demand becomes censored
# note that most summarize methods could be applied row-wise with axis = 1

df_count= df_des.count(axis=1)
for i in range(0, 13):
    # also works: df_grocery.iloc[:,-1].isnull()
    df_des.iloc[i,df_count[i]-1] = np.nan   

df_des

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,NaN,NaN,NaN,NaN,NaN
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,NaN,NaN,NaN
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,NaN,NaN
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,NaN,NaN,NaN,NaN,NaN
6,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,5.0,9.0,11.0,13.0,14.0,16.0,18.0,21.0,23.0,26.0,30.0,33.0,36.0,NaN,NaN
8,2.0,5.0,7.0,8.0,11.0,13.0,16.0,18.0,20.0,21.0,22.0,24.0,27.0,NaN,NaN,NaN
9,1.0,5.0,8.0,9.0,11.0,12.0,15.0,17.0,20.0,22.0,26.0,31.0,37.0,NaN,NaN,NaN


In [10]:
df_des.count(axis=1)

0     11
1      9
2     13
3     14
4      9
5     11
6      9
7     14
8     13
9     13
10     7
11    13
12    14
dtype: int64

In [11]:
# write a function that takes alpha,beta,series as input 
# and produces the traning sample MSE
# allow 3hr for burn-in
def DESMSE(alpha,beta,series):
    return mean_squared_error(series[3:],
                              ExponentialSmoothing(series,trend="add", initialization_method = 'legacy-heuristic').fit(smoothing_level = alpha,smoothing_trend = beta).predict(3,series.size-1))

In [12]:
# the function below allows you to construct a dataframe of alpha-beta combinations
def expand_grid(dictionary):
   return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

# define the alpha and beta values to try as a dictionary
dictionary = {'alpha': np.arange(0.1, 1.1, 0.1), 
              'beta': np.arange(0.1, 1.1, 0.1)}

In [13]:
df_des

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,NaN,NaN,NaN,NaN,NaN
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,NaN,NaN,NaN
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,NaN,NaN
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,NaN,NaN,NaN,NaN,NaN
6,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2.0,5.0,9.0,11.0,13.0,14.0,16.0,18.0,21.0,23.0,26.0,30.0,33.0,36.0,NaN,NaN
8,2.0,5.0,7.0,8.0,11.0,13.0,16.0,18.0,20.0,21.0,22.0,24.0,27.0,NaN,NaN,NaN
9,1.0,5.0,8.0,9.0,11.0,12.0,15.0,17.0,20.0,22.0,26.0,31.0,37.0,NaN,NaN,NaN


#### Sample Code for one row trail

In [14]:
temp = df_des.iloc[0,:].dropna().reset_index(drop=True)
temp

0      2.0
1      6.0
2     10.0
3     12.0
4     16.0
5     18.0
6     22.0
7     23.0
8     28.0
9     32.0
10    37.0
Name: 0, dtype: float64

In [15]:
temp = np.append(temp[0],temp[1:].values - temp[:-1].values)
temp

array([2., 4., 4., 2., 4., 2., 4., 1., 5., 4., 5.])

In [16]:
# construct alpha,beta values to try 
des_results = expand_grid(dictionary)

for alpha in np.arange(0.0, 1.1, 0.1):
    for beta in np.arange(0.0, 1.1, 0.1):
        des_results.loc[(des_results["alpha"] == alpha) & (des_results["beta"] == beta),"mse"] = DESMSE(alpha,beta,temp)

   # find the best (alpha beta): sort and put on row 1
des_results = des_results.sort_values("mse")

In [17]:
predicted_daily_arrival = ExponentialSmoothing(temp,trend="add", initialization_method='legacy-heuristic'). \
                                            fit(smoothing_level = des_results.iloc[0,0],
                                                smoothing_trend = des_results.iloc[0,1]). \
                                          forecast(16-temp.size)

In [18]:
predicted_daily_arrival.cumsum() + list(df_des.iloc[0,:].dropna())[-1]

array([41.21376418, 45.57253467, 50.07631147, 54.72509457, 59.51888399])

In [19]:
df_des.iloc[0,temp.size-1]

37.0

In [20]:
df_des.iloc[0,:temp.size].values

array([ 2.,  6., 10., 12., 16., 18., 22., 23., 28., 32., 37.])

#### For loop for all rows

In [21]:
for row_number in np.arange(0,df_des.shape[0]):
    temp = df_des.iloc[row_number,:].dropna().reset_index(drop=True)
        
# convert cumulative booking into daily booking
# .diff() will work too
# temp[1:] gets booking vlaues for hour 2-16, temp[:-1] gets booking vlaues for hour 1-15,
    temp = np.append(temp[0],temp[1:].values - temp[:-1].values)

    # construct alpha,beta values to try 
    des_results = expand_grid(dictionary)

    for alpha in np.arange(0.1, 1.1, 0.1):
        for beta in np.arange(0.1, 1.1, 0.1):
            des_results.loc[(des_results["alpha"] == alpha) & (des_results["beta"] == beta),"mse"] = DESMSE(alpha,beta,temp)
        
    # find the best (alpha beta): sort and put on row 1
    des_results = des_results.sort_values("mse")


    # use the best (alpha,beta) to forecast for the censored days
    # post results onto df_des
    # forecast(16-temp.size) ensures that we forecast the censored days
    predicted_daily_arrival = ExponentialSmoothing(temp,trend="add", initialization_method='legacy-heuristic'). \
                                            fit(smoothing_level = des_results.iloc[0,0],
                                                smoothing_trend = des_results.iloc[0,1]).forecast(16-temp.size)
                                 

    df_des.iloc[row_number,temp.size:] =  (predicted_daily_arrival.cumsum() + df_des.iloc[row_number,temp.size-1]).round()
  
    #list(df_des.iloc[row_number,:].dropna())[-1]



df_des

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,41.0,46.0,50.0,55.0,60.0
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,39.0,42.0,46.0,50.0,53.0,57.0,60.0
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,41.0,45.0,49.0
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,41.0,43.0
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,40.0,44.0,48.0,52.0,57.0,61.0,65.0
5,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,41.0,45.0,48.0,52.0,55.0
6,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,39.0,43.0,46.0,50.0,54.0,57.0,61.0
7,2.0,5.0,9.0,11.0,13.0,14.0,16.0,18.0,21.0,23.0,26.0,30.0,33.0,36.0,39.0,42.0
8,2.0,5.0,7.0,8.0,11.0,13.0,16.0,18.0,20.0,21.0,22.0,24.0,27.0,29.0,31.0,33.0
9,1.0,5.0,8.0,9.0,11.0,12.0,15.0,17.0,20.0,22.0,26.0,31.0,37.0,43.0,50.0,58.0


### Step 3: summarize total estimated demand for all booking curves as a column onto the original dataset


In [22]:
df_grocery.loc[~df_grocery.isnull().any(axis = 1), 'des'] = df_grocery.loc[~df_grocery.isnull().any(axis = 1), 'Total']

In [23]:
df_grocery.loc[df_grocery.isnull().any(axis = 1), 'des'] = df_des.loc[:,16].values.round()
#list(df_des.loc[:,16].round())

In [24]:
df_grocery.loc[~df_grocery.isnull().any(axis = 1)]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Total,des
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0,31.0
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0,20.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0,31.0
11,1,2,1,1,2,1,3,1,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,23.0,23.0
16,1,3,1,2,3,3,3,1,2.0,3.0,3.0,4.0,4.0,3.0,2.0,1.0,39.0,39.0
18,2,3,4,3,2,1,1,2,3.0,3.0,1.0,4.0,4.0,3.0,2.0,1.0,39.0,39.0
20,1,2,3,2,2,2,1,1,1.0,3.0,2.0,3.0,3.0,2.0,2.0,1.0,31.0,31.0


In [25]:
df_grocery

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Total,des
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN,40.0,60.0
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,60.0
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,40.0,49.0
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN,40.0,43.0
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,65.0
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0,31.0
7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,40.0,55.0
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0,20.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0,31.0
10,2,6,6,3,4,2,4,2,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,61.0


## b. Averaging method estimations

### Step 1: slice grocery curve into segments (in hourly increments)

In [69]:
# Clean the grocery dataset
df_grocery_avg = df_grocery_c.iloc[:,0:16].reset_index()
df_grocery_avg.drop(columns='index', inplace = True)
df_grocery_avg_all = df_grocery_avg.copy()

#### Replace each Censored Demand values from all censored demend curves with NaN 

In [70]:
for i in range(0,len(df_grocery_avg)):
  if df_grocery_avg.isnull().any(axis = 1)[i]:
    df_grocery_avg.iloc[i,len(df_grocery_avg.iloc[i].dropna())-1] = np.nan

df_grocery_avg

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,NaN,NaN,NaN,NaN,NaN
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,NaN,NaN,NaN
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,NaN,NaN
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,4.0,7.0,8.0,11.0,14.0,16.0,17.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0
6,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,NaN,NaN,NaN,NaN,NaN
7,1.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,12.0,13.0,15.0,16.0,17.0,18.0,19.0,20.0
8,1.0,3.0,6.0,8.0,10.0,12.0,14.0,15.0,17.0,19.0,21.0,23.0,25.0,28.0,30.0,31.0
9,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Fill each Nan value from all censored demend curves with Censored Demand values 

In [71]:
for i in range(0,len(df_grocery_avg_all)):
  if df_grocery_avg_all.isnull().any(axis = 1)[i]:
    df_grocery_avg_all.iloc[i,len(df_grocery_avg_all.iloc[i].dropna()):] = df_grocery_avg_all.iloc[i,len(df_grocery_avg_all.iloc[i].dropna())-1]

df_grocery_avg_all

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2.0,6.0,10.0,12.0,16.0,18.0,22.0,23.0,28.0,32.0,37.0,40.0,40.0,40.0,40.0,40.0
1,3.0,7.0,11.0,16.0,20.0,25.0,29.0,31.0,35.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
2,1.0,4.0,6.0,8.0,11.0,17.0,21.0,23.0,26.0,29.0,32.0,36.0,38.0,40.0,40.0,40.0
3,2.0,5.0,9.0,10.0,14.0,17.0,20.0,21.0,24.0,27.0,31.0,33.0,36.0,38.0,40.0,40.0
4,2.0,7.0,12.0,15.0,20.0,25.0,30.0,32.0,36.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
5,1.0,4.0,7.0,8.0,11.0,14.0,16.0,17.0,18.0,20.0,22.0,24.0,26.0,28.0,30.0,31.0
6,3.0,7.0,10.0,12.0,16.0,21.0,26.0,29.0,31.0,35.0,38.0,40.0,40.0,40.0,40.0,40.0
7,1.0,3.0,5.0,6.0,7.0,8.0,10.0,11.0,12.0,13.0,15.0,16.0,17.0,18.0,19.0,20.0
8,1.0,3.0,6.0,8.0,10.0,12.0,14.0,15.0,17.0,19.0,21.0,23.0,25.0,28.0,30.0,31.0
9,2.0,8.0,14.0,17.0,21.0,23.0,27.0,29.0,35.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0


In [72]:
# when leveraging python, we simply segment in hourly increments
# convert dataframe from 'wide' to 'long'
df_avg_all = pd.melt(df_grocery_avg_all.iloc[:,:].reset_index(),id_vars='index',var_name = 'hour',value_name='cum_hourly_sales')
df_avg_all

,index,hour,cum_hourly_sales
0,0,1,2.0
1,1,1,3.0
2,2,1,1.0
3,3,1,2.0
4,4,1,2.0
...,...,...,...
315,15,16,39.0
316,16,16,23.0
317,17,16,39.0
318,18,16,40.0


In [73]:
# sort by curve and hour
# ensure conversion done as expected
df_avg_all.sort_values(by=['index','hour']).head(50)

,index,hour,cum_hourly_sales
0,0,1,2.0
20,0,2,6.0
40,0,3,10.0
60,0,4,12.0
80,0,5,16.0
100,0,6,18.0
120,0,7,22.0
140,0,8,23.0
160,0,9,28.0
180,0,10,32.0


In [74]:
# when leveraging python, we simply segment in hourly increments
# convert dataframe from 'wide' to 'long'
df_avg = pd.melt(df_grocery_avg.iloc[:,:].reset_index(),id_vars='index',var_name = 'hour',value_name='cum_hourly_sales')
df_avg

,index,hour,cum_hourly_sales
0,0,1,2.0
1,1,1,3.0
2,2,1,1.0
3,3,1,2.0
4,4,1,2.0
...,...,...,...
315,15,16,39.0
316,16,16,NaN
317,17,16,39.0
318,18,16,NaN


#### Create the censor column

In [75]:
#create the censor column
df_avg['censor'] = df_avg.cum_hourly_sales.isna()
df_avg

,index,hour,cum_hourly_sales,censor
0,0,1,2.0,False
1,1,1,3.0,False
2,2,1,1.0,False
3,3,1,2.0,False
4,4,1,2.0,False
...,...,...,...,...
315,15,16,39.0,False
316,16,16,NaN,True
317,17,16,39.0,False
318,18,16,NaN,True


### Step 2: Calculate unconstrained arrivals

#### add the censor column from df_avg['censor'] onto df_avg_all

In [76]:
# check which curves are censored, create boolean variable identifying censored cases
# add the censor column from df_avg['censor'] onto df_avg_all
df_avg_all['censor'] = df_avg['censor']
df_avg_all = df_avg_all.sort_values(by=['index','hour'])
df_avg_all.head(50)

,index,hour,cum_hourly_sales,censor
0,0,1,2.0,False
20,0,2,6.0,False
40,0,3,10.0,False
60,0,4,12.0,False
80,0,5,16.0,False
100,0,6,18.0,False
120,0,7,22.0,False
140,0,8,23.0,False
160,0,9,28.0,False
180,0,10,32.0,False


#### hourly_sales diff()

In [77]:
# hourly_sales
# calculates the additional sales arrivals compared to the prior hour, if any
df_avg_all['hourly_sales'] = df_avg_all.groupby('index').cum_hourly_sales.diff()
df_avg_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             320 non-null    int64  
 1   hour              320 non-null    object 
 2   cum_hourly_sales  320 non-null    float64
 3   censor            320 non-null    bool   
 4   hourly_sales      300 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(1)
memory usage: 12.8+ KB


In [78]:
df_avg_all

,index,hour,cum_hourly_sales,censor,hourly_sales
0,0,1,2.0,False,NaN
20,0,2,6.0,False,4.0
40,0,3,10.0,False,4.0
60,0,4,12.0,False,2.0
80,0,5,16.0,False,4.0
...,...,...,...,...,...
239,19,12,23.0,False,3.0
259,19,13,26.0,False,3.0
279,19,14,28.0,False,2.0
299,19,15,30.0,False,2.0


#### Fill the nan values calculated from hourly_sales with the initial value from each demand curve

In [79]:
# Fill the nan values calculated from hourly_sales with the inital value from each demand curve
df_avg_all.loc[pd.isnull(df_avg_all.hourly_sales),'hourly_sales'] = df_avg_all.loc[pd.isnull(df_avg_all.hourly_sales),'cum_hourly_sales']
df_avg_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 320 entries, 0 to 319
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             320 non-null    int64  
 1   hour              320 non-null    object 
 2   cum_hourly_sales  320 non-null    float64
 3   censor            320 non-null    bool   
 4   hourly_sales      320 non-null    float64
dtypes: bool(1), float64(2), int64(1), object(1)
memory usage: 12.8+ KB


#### avg hourly sales across uncensored values of each curve

In [80]:
# avg hourly sales across uncensored values of each curve
temp = df_avg_all[df_avg_all.censor==False].groupby('hour').hourly_sales.mean()
temp


hour
1     1.650000
2     3.450000
3     3.350000
4     2.100000
5     2.850000
6     2.550000
7     3.000000
8     1.631579
9     2.631579
10    2.437500
11    2.500000
12    2.928571
13    2.785714
14    2.400000
15    1.857143
16    1.000000
Name: hourly_sales, dtype: float64

In [81]:
temp = temp.reset_index().rename(columns = {'index' : "hour", 'hourly_sales':'avg_hourly_sales'})
temp

,hour,avg_hourly_sales
0,1,1.650000
1,2,3.450000
2,3,3.350000
3,4,2.100000
4,5,2.850000
5,6,2.550000
6,7,3.000000
7,8,1.631579
8,9,2.631579
9,10,2.437500


#### add the avg_hourly_sales column to df_avg_all using pd.merge

In [82]:
# add the avg_hourly_sales column to df_avg_all using pd.merge
temp['hour'] = temp.hour.astype('object')
df_avg_all = pd.merge(df_avg_all,temp,how='left',on='hour')

In [83]:
# ensure conversion done as expected
df_avg_all.head(40)

,index,hour,cum_hourly_sales,censor,hourly_sales,avg_hourly_sales
0,0,1,2.0,False,2.0,1.650000
1,0,2,6.0,False,4.0,3.450000
2,0,3,10.0,False,4.0,3.350000
3,0,4,12.0,False,2.0,2.100000
4,0,5,16.0,False,4.0,2.850000
5,0,6,18.0,False,2.0,2.550000
6,0,7,22.0,False,4.0,3.000000
7,0,8,23.0,False,1.0,1.631579
8,0,9,28.0,False,5.0,2.631579
9,0,10,32.0,False,4.0,2.437500


#### for the censored rows, replace hourly sales with max(actual,average)

In [84]:
# for the censored rows, replace hourly sales with max(actual,average)
# this calculates the unconstrained arrivals amount for the censored periods
df_avg_all.loc[df_avg_all.censor,'hourly_sales'] = df_avg_all.loc[df_avg_all.censor,['hourly_sales','avg_hourly_sales']].max(axis=1)
df_avg_all.head(20)

,index,hour,cum_hourly_sales,censor,hourly_sales,avg_hourly_sales
0,0,1,2.0,False,2.000000,1.650000
1,0,2,6.0,False,4.000000,3.450000
2,0,3,10.0,False,4.000000,3.350000
3,0,4,12.0,False,2.000000,2.100000
4,0,5,16.0,False,4.000000,2.850000
5,0,6,18.0,False,2.000000,2.550000
6,0,7,22.0,False,4.000000,3.000000
7,0,8,23.0,False,1.000000,1.631579
8,0,9,28.0,False,5.000000,2.631579
9,0,10,32.0,False,4.000000,2.437500


### Step 3: Repeat algorithmic steps until you have reached final demand and report as a column onto the original dataset

In [85]:
df_avg_all['cum_hourly_sales_new'] =  df_avg_all.groupby('index').hourly_sales.cumsum()
df_avg_all.cum_hourly_sales_new 

0       2.0
1       6.0
2      10.0
3      12.0
4      16.0
       ... 
315    23.0
316    26.0
317    28.0
318    30.0
319    31.0
Name: cum_hourly_sales_new, Length: 320, dtype: float64

In [86]:
df_avg_all.groupby('index').cum_hourly_sales_new.tail(1)

15     48.042857
31     53.471429
47     43.257143
63     41.000000
79     53.471429
95     31.000000
111    48.971429
127    20.000000
143    31.000000
159    53.471429
175    23.000000
191    40.000000
207    33.857143
223    42.857143
239    58.540508
255    39.000000
271    26.257143
287    39.000000
303    41.000000
319    31.000000
Name: cum_hourly_sales_new, dtype: float64

In [87]:
df_grocery['avg'] = df_avg_all.groupby('index').cum_hourly_sales_new.tail(1).values.round()

In [88]:
df_grocery

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,Total,des,avg
1,2,4,4,2,4,2,4,1,5.0,4.0,5.0,3.0,NaN,NaN,NaN,NaN,40.0,60.0,48.0
2,3,4,4,5,4,5,4,2,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,60.0,53.0
3,1,3,2,2,3,6,4,2,3.0,3.0,3.0,4.0,2.0,2.0,NaN,NaN,40.0,49.0,43.0
4,2,3,4,1,4,3,3,1,3.0,3.0,4.0,2.0,3.0,2.0,2.0,NaN,40.0,43.0,41.0
5,2,5,5,3,5,5,5,2,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,65.0,53.0
6,1,3,3,1,3,3,2,1,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,31.0,31.0,31.0
7,3,4,3,2,4,5,5,3,2.0,4.0,3.0,2.0,NaN,NaN,NaN,NaN,40.0,55.0,49.0
8,1,2,2,1,1,1,2,1,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,20.0,20.0,20.0
9,1,2,3,2,2,2,2,1,2.0,2.0,2.0,2.0,2.0,3.0,2.0,1.0,31.0,31.0,31.0
10,2,6,6,3,4,2,4,2,6.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,40.0,61.0,53.0


## c. Compare results of AVERAGING and DES method
- Averaging method is "less aggressive" than the DES method

In [89]:
df_grocery[['des','avg']]

,des,avg
1,60.0,48.0
2,60.0,53.0
3,49.0,43.0
4,43.0,41.0
5,65.0,53.0
6,31.0,31.0
7,55.0,49.0
8,20.0,20.0
9,31.0,31.0
10,61.0,53.0


# Part II: Demand Unconstraining: Service products with limited capacity 
* Dataset: booking_data

## a. Implement the Proportional method to apply demand uncensoring to the Hyatt booking data

### STEP 1: calculate the daily cumulative demand

In [90]:
# load hyatt booking data
import pandas as pd
df_booking = pd.read_excel("/content/drive/MyDrive/BA_875/Datasets/booking_data.xlsx")
df_booking.info()
df_booking

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 41 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Thursday      21 non-null     object
 1   Book: day     21 non-null     int64 
 2   Book: day.1   21 non-null     int64 
 3   Book: day.2   21 non-null     int64 
 4   Book: day.3   21 non-null     int64 
 5   Book: day.4   21 non-null     int64 
 6   Book: day.5   21 non-null     int64 
 7   Book: day.6   21 non-null     int64 
 8   Book: day.7   21 non-null     int64 
 9   Book: day.8   21 non-null     int64 
 10  Book: day.9   21 non-null     int64 
 11  Book: day.10  21 non-null     int64 
 12  Book: day.11  21 non-null     int64 
 13  Book: day.12  21 non-null     int64 
 14  Book: day.13  21 non-null     int64 
 15  Book: day.14  21 non-null     int64 
 16  Book: day.15  21 non-null     int64 
 17  Book: day.16  21 non-null     int64 
 18  Book: day.17  21 non-null     int64 
 19  Book: day.

,Thursday,Book: day,Book: day.1,Book: day.2,Book: day.3,Book: day.4,Book: day.5,Book: day.6,Book: day.7,Book: day.8,...,Book: day.30,Book: day.31,Book: day.32,Book: day.33,Book: day.34,Book: day.35,Book: day.36,Book: day.37,Book: day.38,Book: day.39
0,Night #,40,39,38,37,36,35,34,33,32,...,10,9,8,7,6,5,4,3,2,1
1,1,0,0,0,1,1,1,1,1,2,...,16,16,18,18,18,19,20,20,21,23
2,2,0,1,1,1,1,1,1,1,1,...,25,25,25,25,25,25,25,25,25,25
3,3,1,1,2,2,2,2,2,2,3,...,22,25,25,25,25,25,25,25,25,25
4,4,0,0,1,1,1,1,1,1,1,...,11,11,13,14,14,14,14,15,15,16
5,5,0,1,3,4,6,7,9,9,11,...,24,24,25,25,25,25,25,25,25,25
6,6,0,2,4,5,5,5,5,7,7,...,21,21,23,24,25,25,25,25,25,25
7,7,0,0,1,2,2,3,3,3,4,...,18,18,19,21,23,23,23,24,24,25
8,8,0,0,0,0,0,0,0,0,1,...,13,14,15,15,17,17,18,19,19,19
9,9,0,0,0,0,1,2,3,4,5,...,13,13,15,15,16,16,16,18,20,20


In [91]:
# remove first row and first column 
# now each row is a booking curve
df_booking = df_booking.iloc[1:,1:]
df_booking.head(20)

,Book: day,Book: day.1,Book: day.2,Book: day.3,Book: day.4,Book: day.5,Book: day.6,Book: day.7,Book: day.8,Book: day.9,...,Book: day.30,Book: day.31,Book: day.32,Book: day.33,Book: day.34,Book: day.35,Book: day.36,Book: day.37,Book: day.38,Book: day.39
1,0,0,0,1,1,1,1,1,2,2,...,16,16,18,18,18,19,20,20,21,23
2,0,1,1,1,1,1,1,1,1,3,...,25,25,25,25,25,25,25,25,25,25
3,1,1,2,2,2,2,2,2,3,3,...,22,25,25,25,25,25,25,25,25,25
4,0,0,1,1,1,1,1,1,1,2,...,11,11,13,14,14,14,14,15,15,16
5,0,1,3,4,6,7,9,9,11,12,...,24,24,25,25,25,25,25,25,25,25
6,0,2,4,5,5,5,5,7,7,7,...,21,21,23,24,25,25,25,25,25,25
7,0,0,1,2,2,3,3,3,4,5,...,18,18,19,21,23,23,23,24,24,25
8,0,0,0,0,0,0,0,0,1,3,...,13,14,15,15,17,17,18,19,19,19
9,0,0,0,0,1,2,3,4,5,5,...,13,13,15,15,16,16,16,18,20,20
10,0,2,3,3,3,3,3,3,3,3,...,17,17,17,17,17,21,22,23,24,25


In [92]:
# rename columns
import numpy as np
df_booking.columns = np.arange(1,41)
df_booking = df_booking.add_prefix("day")
df_booking.head(20)

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day31,day32,day33,day34,day35,day36,day37,day38,day39,day40
1,0,0,0,1,1,1,1,1,2,2,...,16,16,18,18,18,19,20,20,21,23
2,0,1,1,1,1,1,1,1,1,3,...,25,25,25,25,25,25,25,25,25,25
3,1,1,2,2,2,2,2,2,3,3,...,22,25,25,25,25,25,25,25,25,25
4,0,0,1,1,1,1,1,1,1,2,...,11,11,13,14,14,14,14,15,15,16
5,0,1,3,4,6,7,9,9,11,12,...,24,24,25,25,25,25,25,25,25,25
6,0,2,4,5,5,5,5,7,7,7,...,21,21,23,24,25,25,25,25,25,25
7,0,0,1,2,2,3,3,3,4,5,...,18,18,19,21,23,23,23,24,24,25
8,0,0,0,0,0,0,0,0,1,3,...,13,14,15,15,17,17,18,19,19,19
9,0,0,0,0,1,2,3,4,5,5,...,13,13,15,15,16,16,16,18,20,20
10,0,2,3,3,3,3,3,3,3,3,...,17,17,17,17,17,21,22,23,24,25


#### Replace the censored booking (25) with nan values

In [93]:
# replace the censored booking (25) with nan values
df_booking_cum = df_booking.replace(25,np.nan)
df_booking_cum

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day31,day32,day33,day34,day35,day36,day37,day38,day39,day40
1,0,0,0,1,1,1,1,1,2,2,...,16.0,16.0,18.0,18.0,18.0,19.0,20.0,20.0,21.0,23.0
2,0,1,1,1,1,1,1,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2,2,2,2,2,2,3,3,...,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,1,1,1,1,1,1,1,2,...,11.0,11.0,13.0,14.0,14.0,14.0,14.0,15.0,15.0,16.0
5,0,1,3,4,6,7,9,9,11,12,...,24.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,2,4,5,5,5,5,7,7,7,...,21.0,21.0,23.0,24.0,NaN,NaN,NaN,NaN,NaN,NaN
7,0,0,1,2,2,3,3,3,4,5,...,18.0,18.0,19.0,21.0,23.0,23.0,23.0,24.0,24.0,NaN
8,0,0,0,0,0,0,0,0,1,3,...,13.0,14.0,15.0,15.0,17.0,17.0,18.0,19.0,19.0,19.0
9,0,0,0,0,1,2,3,4,5,5,...,13.0,13.0,15.0,15.0,16.0,16.0,16.0,18.0,20.0,20.0
10,0,2,3,3,3,3,3,3,3,3,...,17.0,17.0,17.0,17.0,17.0,21.0,22.0,23.0,24.0,NaN



### STEP 2: calculate the daily demand to cumulative demand ratio (r)


In [94]:
# create daily demand dataframe
df_booking_d_demand = df_booking.diff(axis =1)
df_booking_d_demand.iloc[:,0]  = df_booking.iloc[:,0]
df_booking_d_demand

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day31,day32,day33,day34,day35,day36,day37,day38,day39,day40
1,0,0,0,1,0,0,0,0,1,0,...,0,0,2,0,0,1,1,0,1,2
2,0,1,0,0,0,0,0,0,0,2,...,1,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,0,0,1,0,...,2,3,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,...,0,0,2,1,0,0,0,1,0,1
5,0,1,2,1,2,1,2,0,2,1,...,1,0,1,0,0,0,0,0,0,0
6,0,2,2,1,0,0,0,2,0,0,...,0,0,2,1,1,0,0,0,0,0
7,0,0,1,1,0,1,0,0,1,1,...,0,0,1,2,2,0,0,1,0,1
8,0,0,0,0,0,0,0,0,1,2,...,1,1,1,0,2,0,1,1,0,0
9,0,0,0,0,1,1,1,1,1,0,...,0,0,2,0,1,0,0,2,2,0
10,0,2,1,0,0,0,0,0,0,0,...,1,0,0,0,0,4,1,1,1,1


In [95]:
df_ratios = round(df_booking_d_demand/ df_booking_cum,3)
df_ratios

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day31,day32,day33,day34,day35,day36,day37,day38,day39,day40
1,NaN,NaN,NaN,1.000,0.000,0.000,0.000,0.000,0.500,0.000,...,0.000,0.000,0.111,0.000,0.000,0.053,0.050,0.000,0.048,0.087
2,NaN,1.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,0.0,0.500,0.000,0.000,0.000,0.000,0.000,0.333,0.000,...,0.091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.500,...,0.000,0.000,0.154,0.071,0.000,0.000,0.000,0.067,0.000,0.062
5,NaN,1.0,0.667,0.250,0.333,0.143,0.222,0.000,0.182,0.083,...,0.042,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,1.0,0.500,0.200,0.000,0.000,0.000,0.286,0.000,0.000,...,0.000,0.000,0.087,0.042,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,1.000,0.500,0.000,0.333,0.000,0.000,0.250,0.200,...,0.000,0.000,0.053,0.095,0.087,0.000,0.000,0.042,0.000,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,0.667,...,0.077,0.071,0.067,0.000,0.118,0.000,0.056,0.053,0.000,0.000
9,NaN,NaN,NaN,NaN,1.000,0.500,0.333,0.250,0.200,0.000,...,0.000,0.000,0.133,0.000,0.062,0.000,0.000,0.111,0.100,0.000
10,NaN,1.0,0.333,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.059,0.000,0.000,0.000,0.000,0.190,0.045,0.043,0.042,NaN


### STEP 3: calculate the average ratios for each day


In [96]:
#Compute the arithmetic mean along the specified axis, ignoring NaNs.Returns the average of the array elements. 
df_average_ratios = np.nanmean(df_ratios, axis=0).round(3)
df_average_ratios

array([1.   , 0.778, 0.444, 0.302, 0.158, 0.134, 0.085, 0.127, 0.248,
       0.157, 0.148, 0.134, 0.11 , 0.071, 0.066, 0.071, 0.078, 0.058,
       0.082, 0.037, 0.036, 0.071, 0.072, 0.066, 0.037, 0.033, 0.055,
       0.044, 0.061, 0.031, 0.033, 0.023, 0.064, 0.03 , 0.038, 0.04 ,
       0.034, 0.042, 0.022, 0.034])

### STEP 4: calculate R (cumulative/total) and Q (daily/total)


In [97]:
# applying initial values for R_m and Q_m before recursive calculations
# R_m = 1
# Q_m = r_m, where m is the last day
array_R = np.ones(len(df_average_ratios))
array_Q = np.ones(len(df_average_ratios))
# Q_40 = r_40
array_Q[39] = df_average_ratios[39]

In [98]:
# calculate the R and Q backwardly
# remember that numpy array index starts from 0. so index = 39 is for day = 40
# applying formulas to fill out remaining values
# R_i-1 = R_i - Q_i
# Q_i-1 = r_i-1 * R_i-1
for j in range(len(df_average_ratios)-2,-1,-1): 
    array_R[j] = array_R[j+1] - array_Q[j+1]
    array_Q[j] = df_average_ratios[j]*array_R[j]

In [99]:
array_R

array([0.00532064, 0.02396683, 0.0431058 , 0.06175616, 0.07334461,
       0.08469355, 0.09256125, 0.10602664, 0.14099287, 0.16725132,
       0.19630437, 0.22667941, 0.25469597, 0.27416143, 0.29353472,
       0.31596848, 0.34269901, 0.36379937, 0.39629561, 0.41152192,
       0.42688996, 0.45951556, 0.49516763, 0.53015807, 0.55052759,
       0.56931498, 0.60244972, 0.63017753, 0.67111558, 0.69258574,
       0.71622103, 0.73308191, 0.78320717, 0.80743007, 0.8393244 ,
       0.87429625, 0.90506858, 0.944748  , 0.966     , 1.        ])

In [100]:
array_Q

array([0.00532064, 0.01864619, 0.01913898, 0.01865036, 0.01158845,
       0.01134894, 0.00786771, 0.01346538, 0.03496623, 0.02625846,
       0.02905305, 0.03037504, 0.02801656, 0.01946546, 0.01937329,
       0.02243376, 0.02673052, 0.02110036, 0.03249624, 0.01522631,
       0.01536804, 0.03262561, 0.03565207, 0.03499043, 0.02036952,
       0.01878739, 0.03313473, 0.02772781, 0.04093805, 0.02147016,
       0.02363529, 0.01686088, 0.05012526, 0.0242229 , 0.03189433,
       0.03497185, 0.03077233, 0.03967942, 0.021252  , 0.034     ])

### STEP 5: summarize total estimated demand for all booking curves as a column onto the original dataset (unconstrained total demand = last unconstrainted cumulative demand / R ratio)

In [101]:
df_booking

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day31,day32,day33,day34,day35,day36,day37,day38,day39,day40
1,0,0,0,1,1,1,1,1,2,2,...,16,16,18,18,18,19,20,20,21,23
2,0,1,1,1,1,1,1,1,1,3,...,25,25,25,25,25,25,25,25,25,25
3,1,1,2,2,2,2,2,2,3,3,...,22,25,25,25,25,25,25,25,25,25
4,0,0,1,1,1,1,1,1,1,2,...,11,11,13,14,14,14,14,15,15,16
5,0,1,3,4,6,7,9,9,11,12,...,24,24,25,25,25,25,25,25,25,25
6,0,2,4,5,5,5,5,7,7,7,...,21,21,23,24,25,25,25,25,25,25
7,0,0,1,2,2,3,3,3,4,5,...,18,18,19,21,23,23,23,24,24,25
8,0,0,0,0,0,0,0,0,1,3,...,13,14,15,15,17,17,18,19,19,19
9,0,0,0,0,1,2,3,4,5,5,...,13,13,15,15,16,16,16,18,20,20
10,0,2,3,3,3,3,3,3,3,3,...,17,17,17,17,17,21,22,23,24,25


In [103]:
# All the uncensored booking curve row index
df_booking_cum[~df_booking_cum.isna().any(axis = 1)].index

Int64Index([1, 4, 8, 9, 11, 15, 17, 20], dtype='int64')

In [104]:
# All the censored booking curve row index
df_booking_cum[df_booking_cum.isna().any(axis = 1)].index

Int64Index([2, 3, 5, 6, 7, 10, 12, 13, 14, 16, 18, 19], dtype='int64')

In [111]:
# For the censored booking curve row index, fill the ratio (last uncensored cumulative booking value / R) into the 'prop' column
# R = cumulative demand / total demand
# Q = daily demand / total demand
# r = daily demand / cumulative demand

for i in df_booking_cum[df_booking_cum.isna().any(axis = 1)].index:

  df_booking.loc[i,'last cumulative demand'] = df_booking_cum.loc[i].dropna()[-1]


  df_booking.loc[i,'R'] =  array_R[len(df_booking_cum.loc[i].dropna()) - 1]

  df_booking.loc[i,'prop'] =  (df_booking.loc[i,'last cumulative demand'] /  df_booking.loc[i,'R'] ).round()
  
  #(df_booking_cum.loc[i].dropna()[-1] / array_R[len(df_booking_cum.loc[i].dropna()) - 1]).round()
  


# For the uncensored booking curve row index, fill the 'day40' value into the 'prop' column
for i in df_booking_cum[~df_booking_cum.isna().any(axis = 1)].index:

   df_booking.loc[i,'last cumulative demand'] = df_booking_cum.loc[i].dropna()[-1]

   df_booking.loc[i,'R'] =  array_R[len(df_booking_cum.loc[i].dropna()) - 1]

   df_booking.loc[i,'prop'] = df_booking.loc[i,'last cumulative demand'] / df_booking.loc[i,'R'] 
   
   #df_booking_cum[~df_booking_cum.isna().any(axis = 1)].loc[i,'day40']
   


In [112]:
df_booking.iloc[:, -3:]

,last cumulative demand,R,prop
1,23.0,1.000000,23.0
2,24.0,0.692586,35.0
3,22.0,0.716221,31.0
4,16.0,1.000000,16.0
5,24.0,0.733082,33.0
6,24.0,0.807430,30.0
7,24.0,0.966000,25.0
8,19.0,1.000000,19.0
9,20.0,1.000000,20.0
10,24.0,0.966000,25.0
